## Notebook to demonstrate Data-Services workflow
### The workflow in a nutshell
TAO Data Services include 4 key pipelines:
1. Offline data augmentation using DALI
2. Auto labeling using TAO Mask Auto-labeler (MAL)
3. Annotation conversion
4. Groundtruth analytics

## Learning Objectives

In this notebook, you will learn how to leverage the simplicity and convenience of TAO to:

* Convert KITTI dataset to COCO format
* Run auto-labeling to generate pseudo masks for KITTI bounding boxes
* Apply data augmentation to the KITTI dataset with bounding boxe refinement
* Run data analytics to collect useful statistics on the original and augmented KITTI dataset

### Table of contents

1. [Convert KITTI data to COCO format](#head-1)
2. [Generate pseudo-masks with the auto-labeler](#head-2)
3. [Apply data augmentation](#head-3)
4. [Perform data analytics](#head-4)


### Requirements
Please find the server requirements [here](https://docs.nvidia.com/tao/tao-toolkit/text/tao_toolkit_api/api_setup.html#)

In [ ]:
# imports
import json
import os
import requests
import time
from IPython.display import clear_output

### FIXME

1. Assign a workdir in FIXME 1
1. Assign the ip_address and port_number in FIXME 2 ([info](https://docs.nvidia.com/tao/tao-toolkit/text/tao_toolkit_api/api_rest_api.html))
1. Assign the ngc_api_key variable in FIXME 3
1. Assign path to save the dataset in FIXME 4

In [ ]:
workdir = "workdir_data_services"  # FIXME 1
host_url = "http://<ip_address>:<port_number>"  # FIXME 2 example: https://10.137.149.22:32334
# In host machine, node ip_address and port_number can be obtained as follows,
# ip_address: hostname -I
# port_number: kubectl get service ingress-nginx-controller -o jsonpath='{.spec.ports[0].nodePort}'
ngc_api_key = "<ngc_api_key>"  # FIXME 3 example: (Add NGC API key) 

In [ ]:
data_dir = "dataset_path"    # FIXME 4
job_map = {}

In [ ]:
# Exchange NGC_API_KEY for JWT
data = json.dumps({"ngc_api_key": ngc_api_key})
response = requests.post(f"{host_url}/api/v1/login", data=data)
user_id = response.json()["user_id"]
print("User ID", user_id)
token = response.json()["token"]
print("JWT", token)

# Set base URL
base_url = f"{host_url}/api/v1/users/{user_id}"
print("API Calls will be forwarded to", base_url)

headers = {"Authorization": f"Bearer {token}"}

In [ ]:
# Creating workdir
if not os.path.isdir(workdir):
    os.makedirs(workdir)

### Function to split tar files <a class="anchor" id="head-1.1"></a>

In [ ]:
import os
import tarfile

def split_tar_file(input_tar_path, output_dir, max_split_size=0.2*1024*1024*1024):
	os.makedirs(output_dir, exist_ok=True)
	
	with tarfile.open(input_tar_path, 'r') as original_tar:
		members = original_tar.getmembers()
		current_split_size = 0
		current_split_number = 0
		current_split_name = os.path.join(output_dir, f'smaller_file_{current_split_number}.tar')
		
		with tarfile.open(current_split_name, 'w') as split_tar:
			for member in members:
				if current_split_size + member.size <= max_split_size:
					split_tar.addfile(member, original_tar.extractfile(member))
					current_split_size += member.size
				else:
					split_tar.close()
					current_split_number += 1
					current_split_name = os.path.join(output_dir, f'smaller_file_{current_split_number}.tar')
					current_split_size = 0
					split_tar = tarfile.open(current_split_name, 'w')  # Open a new split tar archive
					split_tar.addfile(member, original_tar.extractfile(member))
					current_split_size += member.size

## 1. Convert KITTI data to COCO format <a class="anchor" id="head-1"></a>
We would first convert the dataset from KITTI to COCO formats.

### Create the dataset
We support both KITTI and COCO data formats

KITTI dataset follow the directory structure displayed below:
```
$DATA_DIR/dataset
├── images
│   ├── image_name_1.jpg
│   ├── image_name_2.jpg
|   ├── ...
└── labels
    ├── image_name_1.txt
    ├── image_name_2.txt
    ├── ...
```

And COCO dataset follow the directory structure displayed below:
```
$DATA_DIR/dataset
├── images
│   ├── image_name_1.jpg
│   ├── image_name_2.jpg
|   ├── ...
└── annotations.json
```
For this notebook, we will be using the kitti object detection dataset for this example. To find more details, please visit [here](http://www.cvlibs.net/datasets/kitti/eval_object.php?obj_benchmark=2d).

### Assigning the task and action

In [ ]:
# Defining the task
network_arch = "annotations"
action = "convert"

data = json.dumps({"network_arch": network_arch})

endpoint = f"{base_url}/experiments"


response = requests.post(endpoint, data=data, headers=headers)
print(response.json())

annotation_conversion_experiment_id = response.json()["id"]
print(annotation_conversion_experiment_id)

In [ ]:
# List tasks
endpoint = f"{base_url}/experiments"

response = requests.get(endpoint, headers=headers)

print(response)
print("model id\t\t\t     network architecture")
for rsp in response.json():
    print(rsp["id"],rsp["network_arch"])

In [ ]:
# Dataset Links
images_url = "https://s3.eu-central-1.amazonaws.com/avg-kitti/data_object_image_2.zip"
labels_url = "https://s3.eu-central-1.amazonaws.com/avg-kitti/data_object_label_2.zip"

In [ ]:
# Download the dataset
!wget -O images.zip {images_url}
!wget -O labels.zip {labels_url}

In [ ]:
!unzip -q images.zip -d {data_dir}/
!unzip -q labels.zip -d {data_dir}/
!mkdir -p {data_dir}/images {data_dir}/labels
!mv {data_dir}/training/image_2/000* {data_dir}/images/
!mv {data_dir}/training/label_2/000* {data_dir}/labels/
!cd {data_dir} && tar -cf kitti_dataset.tar images labels
!rm -rf images.zip labels.zip {data_dir}/training/ {data_dir}/training/ {data_dir}/testing/

In [ ]:
# Create Dataset
ds_type = "object_detection"
ds_format = "kitti"
data = json.dumps({"type": ds_type, "format": ds_format})

endpoint = f"{base_url}/datasets"

response = requests.post(endpoint, data=data, headers=headers)

print(response)
print(response.json())
kitti_dataset_id = response.json()["id"]

In [ ]:
# Update
dataset_information = {"name": "Dataset",
                       "description": "My dataset"}
data = json.dumps(dataset_information)

endpoint = f"{base_url}/datasets/{kitti_dataset_id}"

response = requests.patch(endpoint, data=data, headers=headers)

print(response)
print(response.json())

In [ ]:
# Upload
dataset_path = f"{data_dir}/kitti_dataset.tar"
output_dir = os.path.join(os.path.dirname(os.path.abspath(dataset_path)), network_arch, "test")
split_tar_file(dataset_path, output_dir)
for idx, tar_dataset_path in enumerate(os.listdir(output_dir)):
    print(f"Uploading {idx+1}/{len(os.listdir(output_dir))} tar split")
    files = [("file", open(os.path.join(output_dir, tar_dataset_path),"rb"))]

    endpoint = f"{base_url}/datasets/{kitti_dataset_id}:upload"

    response = requests.post(endpoint, files=files, headers=headers)

    print(response)
    print(response.json())

### List the created datasets

In [ ]:
# List the created datasets
endpoint = f"{base_url}/datasets"

response = requests.get(endpoint, headers=headers)

print(response)
print("id\t\t\t\t\t type\t\t\t format\t\t name")
for rsp in response.json():
    print(f"{rsp['id']}\t{rsp['type']}\t{rsp['format']}\t\t{rsp['name']}")

### Assign the dataset


In [ ]:
# Assign Dataset
dataset_information = {"inference_dataset": kitti_dataset_id}

data = json.dumps(dataset_information)

endpoint = f"{base_url}/experiments/{annotation_conversion_experiment_id}"

response = requests.patch(endpoint, data=data, headers=headers)

print(response)
print(response.json())

### Define the specs


In [ ]:
# Get default spec schema
endpoint = f"{base_url}/experiments/{annotation_conversion_experiment_id}/specs/{action}/schema"
 
response = requests.get(endpoint, headers=headers)

print(response)
annotations_conversion_specs = response.json()["default"]
print(json.dumps(annotations_conversion_specs, sort_keys=True, indent=4))

In [ ]:
# Updating spec file
annotations_conversion_specs["data"]["input_format"] = "KITTI"
annotations_conversion_specs["data"]["output_format"] = "COCO"
print(json.dumps(annotations_conversion_specs, sort_keys=True, indent=4))

### Execute the data format conversion action 


In [ ]:
# Run action
parent = None
data = json.dumps({"parent_job_id":parent, "action":action, "specs":annotations_conversion_specs})

endpoint = f"{base_url}/experiments/{annotation_conversion_experiment_id}/jobs"

response = requests.post(endpoint, data=data, headers=headers)

print(response)
print(response.json())

job_map[action] = response.json()
print(job_map)

In [ ]:
# Monitor job status by repeatedly running this cell
convert_job_id = job_map[action]
endpoint = f"{base_url}/experiments/{annotation_conversion_experiment_id}/jobs/{convert_job_id}"

while True:
    clear_output(wait=True)
    response = requests.get(endpoint, headers=headers)
    
    if "error_desc" in response.json().keys() and response.json()["error_desc"] in ("Job trying to retrieve not found", "No AutoML run found"):
        print("Job is being created")
        time.sleep(5)
        continue
    print(response)
    print(json.dumps(response.json(), sort_keys=True, indent=4))
       
    if response.json().get("status") in ["Done","Error", "Canceled"] or response.status_code not in (200,201):
        break
    time.sleep(15)

In [ ]:
# Download job contents
convert_job_id = job_map[action]
endpoint = f"{base_url}/experiments/{annotation_conversion_experiment_id}/jobs/{convert_job_id}"
response = requests.get(endpoint, headers=headers)
assert response.status_code in (200, 201)
expected_file_size = response.json().get("job_tar_stats", {}).get("file_size")
print("expected_file_size: ", expected_file_size)

endpoint = f'{base_url}/experiments/{annotation_conversion_experiment_id}/jobs/{convert_job_id}:download'
temptar = f'{convert_job_id}.tar.gz'

while True:
    # Check if the file already exists
    headers_download_job = dict(headers)
    if os.path.exists(temptar):
        # Get the current file size
        file_size = os.path.getsize(temptar)

        # If the file size matches the expected size, break out of the loop
        if file_size >= expected_file_size:
            print("Download completed successfully.")
            print("Untarring")
            # Untar to destination
            tar_command = f'tar -xf {temptar} -C {workdir}/'
            os.system(tar_command)
            os.remove(temptar)
            print(f"Results at {workdir}/{convert_job_id}")
            convert_out_path = f"{workdir}/{convert_job_id}"
            break

        # Set the headers to resume the download from where it left off
        headers_download_job['Range'] = f'bytes={file_size}-'
    # Open the file for writing in binary mode
    with open(temptar, 'ab') as f:
        try:
            response = requests.get(endpoint, headers=headers_download_job, stream=True)
            # Check if the request was successful
            if response.status_code in [200, 206]:
                # Iterate over the content in chunks
                for chunk in response.iter_content(chunk_size=1024):
                    if chunk:
                        # Write the chunk to the file
                        f.write(chunk)
                        # Flush and sync the file to disk
                        f.flush()
                        os.fsync(f.fileno())
            else:
                print(f"Failed to download file. Status code: {response.status_code}")
        except requests.exceptions.RequestException as e:
            print("Connection interrupted during download, resuming download from breaking point")
            time.sleep(5)  # Sleep for a while before retrying the request
            continue  # Continue the loop to retry the request


## 2. Generate pseudo-masks with the auto-labeler <a class="anchor" id="head-2"></a>
Here we will use a pretrained MAL model to generate pseudo-masks for the converted KITTI data. 

### Create the model

In [ ]:
# Defining the task
network_arch = "auto_label"
action = "generate"

data = json.dumps({"network_arch": network_arch})
print(data)
endpoint = f"{base_url}/experiments"

response = requests.post(endpoint, data=data, headers=headers)

print(response)
print(response.json())
pseudo_mask_experiment_id = response.json()["id"]
print(pseudo_mask_experiment_id)

In [ ]:
# Reformatting the dataset
# Untar to destination
tar_command = f'mkdir -p {workdir}/{convert_job_id}_coco/ && tar -xf {dataset_path} -C {workdir}/{convert_job_id}_coco/'
os.system(tar_command)

# Copy the annotations
copy_command = f'cp {convert_out_path}/{kitti_dataset_id}.json {workdir}/{convert_job_id}_coco/annotations.json'
os.system(copy_command)

# Tar the dataset
tar_command = f'cd {workdir} && tar -cf {convert_job_id}_coco.tar {convert_job_id}_coco'
os.system(tar_command)
coco_data = f'{workdir}/{convert_job_id}_coco.tar'

### Create the dataset
We would be formatting the original dataset to include the COCO annotations generated.

In [ ]:
# Create Dataset
ds_type = "object_detection"
ds_format = "coco"
data = json.dumps({"type": ds_type,"format": ds_format})

endpoint = f"{base_url}/datasets"

response = requests.post(endpoint, data=data, headers=headers)

print(response)
print(response.json())
coco_dataset_id = response.json()["id"]

In [ ]:
# Update dataset information
dataset_information = {"name": "Dataset",
                       "description": "My dataset"}
data = json.dumps(dataset_information)

endpoint = f"{base_url}/datasets/{coco_dataset_id}"

response = requests.patch(endpoint, data=data, headers=headers)

print(response)
print(response.json())

In [ ]:
# Upload
output_dir = os.path.join(os.path.dirname(os.path.abspath(coco_data)), network_arch, "test")
split_tar_file(coco_data, output_dir)
for idx, tar_dataset_path in enumerate(os.listdir(output_dir)):
    print(f"Uploading {idx+1}/{len(os.listdir(output_dir))} tar split")
    files = [("file",open(os.path.join(output_dir, tar_dataset_path), "rb"))]

    endpoint = f"{base_url}/datasets/{coco_dataset_id}:upload"

    response = requests.post(endpoint, files=files, headers=headers)

    print(response)
    print(response.json())

### Find the PTM

In [ ]:
# List models
endpoint = f"{base_url}/experiments"

response = requests.get(endpoint, headers=headers)

print(response)
print("model id\t\t\t     network architecture")
for rsp in response.json():
    if rsp["name"] == "Mask Auto Label":
        print(f'PTM Name: {rsp["name"]}; PTM version: {rsp["version"]}; NGC PATH: {rsp["ngc_path"]}; Additional info: {rsp["additional_id_info"]}')

In [ ]:
pretrained_map = {"auto_label" : "mask_auto_label:trainable_v1.0"}

In [ ]:
# Get pretrained model
model_list = f"{base_url}/experiments"
response = requests.get(model_list, headers=headers)

response_json = response.json()

# Search for ptm with given ngc path
ptm = []
for rsp in response_json:
    if rsp["network_arch"] == network_arch and rsp["ngc_path"].endswith(pretrained_map[network_arch]):
        ptm_id = rsp["id"]
        ptm = [ptm_id]
        print("Metadata for model with requested NGC Path")
        print(rsp)
        break

### Assign the dataset

In [ ]:
# Assign Dataset
dataset_information = {"inference_dataset": coco_dataset_id}

data = json.dumps(dataset_information)

endpoint = f"{base_url}/experiments/{pseudo_mask_experiment_id}"

response = requests.patch(endpoint, data=data, headers=headers)

print(response)
print(response.json())

### Assign the PTM

In [ ]:
# Assign PTM
dataset_information = {"base_experiment": ptm}

data = json.dumps(dataset_information)

endpoint = f"{base_url}/experiments/{pseudo_mask_experiment_id}"

response = requests.patch(endpoint, data=data, headers=headers)

print(response)
print(response.json())

### Define the specs


In [ ]:
# Get default spec schema
endpoint = f"{base_url}/experiments/{pseudo_mask_experiment_id}/specs/{action}/schema"

response = requests.get(endpoint, headers=headers)

print(response)
auto_label_generate_specs = response.json()["default"]
print(json.dumps(auto_label_generate_specs, sort_keys=True, indent=4))

In [ ]:
# Override any of the parameters listed in the previous cell as required
auto_label_generate_specs["gpu_ids"] = [0]
print(json.dumps(auto_label_generate_specs, sort_keys=True, indent=4))

### Execute the auto labeling action

In [ ]:
# Run action
parent = None

data = json.dumps({"parent_job_id": parent, "action":action, "specs":auto_label_generate_specs})

endpoint = f"{base_url}/experiments/{pseudo_mask_experiment_id}/jobs"

response = requests.post(endpoint, data=data, headers=headers)

print(response)
print(response.json())

job_map[action] = response.json()
print(job_map)

In [ ]:
# Monitor job status by repeatedly running this cell
label_job_id = job_map[action]
endpoint = f"{base_url}/experiments/{pseudo_mask_experiment_id}/jobs/{label_job_id}"

while True: 
    clear_output(wait=True)
    response = requests.get(endpoint, headers=headers)
    
    if "error_desc" in response.json().keys() and response.json()["error_desc"] in ("Job trying to retrieve not found", "No AutoML run found"):
        print("Job is being created")
        time.sleep(5)
        continue
    print(response)
    print(json.dumps(response.json(), sort_keys=True, indent=4))

    if response.json().get("status") in ["Done","Error", "Canceled"] or response.status_code not in (200,201):
        break
    time.sleep(15)

In [ ]:
# Download job contents
label_job_id = job_map[action]

endpoint = f"{base_url}/experiments/{pseudo_mask_experiment_id}/jobs/{label_job_id}"
response = requests.get(endpoint, headers=headers)
assert response.status_code in (200, 201)
expected_file_size = response.json().get("job_tar_stats", {}).get("file_size")
print("expected_file_size: ", expected_file_size)

endpoint = f'{base_url}/experiments/{pseudo_mask_experiment_id}/jobs/{label_job_id}:download'
# Save
temptar = f'{label_job_id}.tar.gz'

while True:
    # Check if the file already exists
    headers_download_job = dict(headers)
    if os.path.exists(temptar):
        # Get the current file size
        file_size = os.path.getsize(temptar)

        # If the file size matches the expected size, break out of the loop
        if file_size >= expected_file_size:
            print("Download completed successfully.")
            print("Untarring")
            # Untar to destination
            tar_command = f'tar -xf {temptar} -C {workdir}/'
            os.system(tar_command)
            os.remove(temptar)
            print(f"Results at {workdir}/{label_job_id}")
            inference_out_path = f"{workdir}/{label_job_id}"
            break

        # Set the headers to resume the download from where it left off
        headers_download_job['Range'] = f'bytes={file_size}-'
    # Open the file for writing in binary mode
    with open(temptar, 'ab') as f:
        try:
            response = requests.get(endpoint, headers=headers_download_job, stream=True)
            # Check if the request was successful
            if response.status_code in [200, 206]:
                # Iterate over the content in chunks
                for chunk in response.iter_content(chunk_size=1024):
                    if chunk:
                        # Write the chunk to the file
                        f.write(chunk)
                        # Flush and sync the file to disk
                        f.flush()
                        os.fsync(f.fileno())
            else:
                print(f"Failed to download file. Status code: {response.status_code}")
        except requests.exceptions.RequestException as e:
            print("Connection interrupted during download, resuming download from breaking point")
            time.sleep(5)  # Sleep for a while before retrying the request
            continue  # Continue the loop to retry the request

## 3. Apply data augmentation <a class="anchor" id="head-3"></a>
In this section, we run offline augmentation with the original dataset. During the augmentation process, we can use the pseudo-masks generated from the last step to refine the distorted or rotated bounding boxes

### Assigning the task and action

In [ ]:
# Defining the task
network_arch = "augmentation"
action = "generate"

data = json.dumps({"network_arch": network_arch})

endpoint = f"{base_url}/experiments"

response = requests.post(endpoint, data=data, headers=headers)

data_aug_experiment_id = response.json()["id"]
print(data_aug_experiment_id)

In [ ]:
# List tasks
endpoint = f"{base_url}/experiments"

response = requests.get(endpoint, headers=headers)

print(response)
print("model id\t\t\t     network architecture")
for rsp in response.json():
    print(rsp["id"],rsp["network_arch"])

### Create the dataset
We would be formatting the dataset to include the generated mask information.

In [ ]:
# Format the dataset
copy_command = f'cp {workdir}/{label_job_id}/label.json {workdir}/{convert_job_id}_coco'
os.system(copy_command)

# Tar the dataset
tar_command = f'cd {workdir} && tar -cvf {label_job_id}_coco.tar {convert_job_id}_coco'
os.system(tar_command)
coco_data = f'{workdir}/{label_job_id}_coco.tar'

In [ ]:
# Create Dataset
ds_type = "object_detection"
ds_format = "coco"
data = json.dumps({"type": ds_type,"format": ds_format})

endpoint = f"{base_url}/datasets"

response = requests.post(endpoint, data=data, headers=headers)

print(response)
print(response.json())
coco_dataset_augment_id = response.json()["id"]

In [ ]:
# Update
dataset_information = {"name": "Dataset",
                       "description": "My dataset"}
data = json.dumps(dataset_information)

endpoint = f"{base_url}/datasets/{coco_dataset_augment_id}"

response = requests.patch(endpoint, data=data, headers=headers)

print(response)
print(response.json())

In [ ]:
# Upload
output_dir = os.path.join(os.path.dirname(os.path.abspath(coco_data)), network_arch, "test")
split_tar_file(coco_data, output_dir)
for idx, tar_dataset_path in enumerate(os.listdir(output_dir)):
    print(f"Uploading {idx+1}/{len(os.listdir(output_dir))} tar split")
    files = [("file",open(os.path.join(output_dir, tar_dataset_path), "rb"))]

    endpoint = f"{base_url}/datasets/{coco_dataset_augment_id}:upload"

    response = requests.post(endpoint, files=files, headers=headers)

    print(response)
    print(response.json())

### Assign the dataset


In [ ]:
# Assign Dataset
dataset_information = {"inference_dataset": coco_dataset_augment_id}

data = json.dumps(dataset_information)

endpoint = f"{base_url}/experiments/{data_aug_experiment_id}"

response = requests.patch(endpoint, data=data, headers=headers)

print(response)
print(response.json())

### Define the specs


In [ ]:
# Get default spec schema
endpoint = f"{base_url}/experiments/{data_aug_experiment_id}/specs/{action}/schema"

response = requests.get(endpoint, headers=headers)

print(response)
augmentation_generate_specs = response.json()["default"]
print(json.dumps(augmentation_generate_specs, sort_keys=True, indent=4))

In [ ]:
# Make changes to the specs if necessary
print(json.dumps(augmentation_generate_specs, sort_keys=True, indent=4))

### Execute the data augmentation action


In [ ]:
# Run action
parent = None

data = json.dumps({"parent_job_id":parent, "action":action, "specs":augmentation_generate_specs})

endpoint = f"{base_url}/experiments/{data_aug_experiment_id}/jobs"

response = requests.post(endpoint, data=data, headers=headers)

print(response)
print(response.json())

job_map[action] = response.json()
print(job_map)

In [ ]:
# Monitor job status by repeatedly running this cell
augment_job_id = job_map[action]
endpoint = f"{base_url}/experiments/{data_aug_experiment_id}/jobs/{augment_job_id}"

while True:
    clear_output(wait=True)
    response = requests.get(endpoint, headers=headers)
    
    if "error_desc" in response.json().keys() and response.json()["error_desc"] in ("Job trying to retrieve not found", "No AutoML run found"):
        print("Job is being created")
        time.sleep(5)
        continue
    print(response)
    print(json.dumps(response.json(), sort_keys=True, indent=4))

    if response.json().get("status") in ["Done","Error", "Canceled"] or response.status_code not in (200,201):
        break
    time.sleep(15)

In [ ]:
# Download job contents
augment_job_id = job_map[action]
temptar = f'{augment_job_id}.tar.gz'

endpoint = f"{base_url}/experiments/{data_aug_experiment_id}/jobs/{augment_job_id}"
response = requests.get(endpoint, headers=headers)
assert response.status_code in (200, 201)
expected_file_size = response.json().get("job_tar_stats", {}).get("file_size")
print("expected_file_size: ", expected_file_size)

endpoint = f'{base_url}/experiments/{data_aug_experiment_id}/jobs/{augment_job_id}:download'

while True:
    # Check if the file already exists
    headers_download_job = dict(headers)
    if os.path.exists(temptar):
        # Get the current file size
        file_size = os.path.getsize(temptar)

        # If the file size matches the expected size, break out of the loop
        if file_size >= expected_file_size:
            print("Download completed successfully.")
            print("Untarring")
            # Untar to destination
            tar_command = f'tar -xf {temptar} -C {workdir}/'
            os.system(tar_command)
            os.remove(temptar)
            print(f"Results at {workdir}/{augment_job_id}")
            inference_out_path = f"{workdir}/{augment_job_id}"
            break

        # Set the headers to resume the download from where it left off
        headers_download_job['Range'] = f'bytes={file_size}-'
    # Open the file for writing in binary mode
    with open(temptar, 'ab') as f:
        try:
            response = requests.get(endpoint, headers=headers_download_job, stream=True)
            # Check if the request was successful
            if response.status_code in [200, 206]:
                # Iterate over the content in chunks
                for chunk in response.iter_content(chunk_size=1024):
                    if chunk:
                        # Write the chunk to the file
                        f.write(chunk)
                        # Flush and sync the file to disk
                        f.flush()
                        os.fsync(f.fileno())
            else:
                print(f"Failed to download file. Status code: {response.status_code}")
        except requests.exceptions.RequestException as e:
            print("Connection interrupted during download, resuming download from breaking point")
            time.sleep(5)  # Sleep for a while before retrying the request
            continue  # Continue the loop to retry the request

## 4. Perform data analytics  <a class="anchor" id="head-4"></a>
Next, we perform analytics with the KITTI dataset.

### Assigning the task and action

In [ ]:
# Defining the task
network_arch = "analytics"
action = "analyze"

data = json.dumps({"network_arch": network_arch})

endpoint = f"{base_url}/experiments"

response = requests.post(endpoint, data=data, headers=headers)

data_analytics_experiment_id = response.json()["id"]
print(data_analytics_experiment_id)

In [ ]:
# List tasks
endpoint = f"{base_url}/experiments"

response = requests.get(endpoint, headers=headers)

print(response)
print("model id\t\t\t     network architecture")
for rsp in response.json():
    print(rsp["id"],rsp["network_arch"])

### Assign the dataset


In [ ]:
# Assign Dataset
dataset_information = {"inference_dataset": kitti_dataset_id}

data = json.dumps(dataset_information)

endpoint = f"{base_url}/experiments/{data_analytics_experiment_id}"

response = requests.patch(endpoint, data=data, headers=headers)

print(response)
print(response.json())

### Define the specs


In [ ]:
# Get default spec schema
endpoint = f"{base_url}/experiments/{data_analytics_experiment_id}/specs/{action}/schema"
 
response = requests.get(endpoint, headers=headers)

print(response)
analytics_analyze_specs = response.json()["default"]
print(json.dumps(analytics_analyze_specs, sort_keys=True, indent=4))

In [ ]:
# Make changes to the specs if necessary
print(json.dumps(analytics_analyze_specs, sort_keys=True, indent=4))

### Execute the data analytics action


In [ ]:
# Run action
parent = None

data = json.dumps({"parent_job_id":parent, "action":action, "specs":analytics_analyze_specs})

endpoint = f"{base_url}/experiments/{data_analytics_experiment_id}/jobs"

response = requests.post(endpoint, data=data, headers=headers)

print(response)
print(response.json())

job_map[action] = response.json()
print(job_map)

In [ ]:
# Monitor job status by repeatedly running this cell
job_id = job_map[action]
endpoint = f"{base_url}/experiments/{data_analytics_experiment_id}/jobs/{job_id}"

while True:
    clear_output(wait=True)
    response = requests.get(endpoint, headers=headers)
    
    if "error_desc" in response.json().keys() and response.json()["error_desc"] in ("Job trying to retrieve not found", "No AutoML run found"):
        print("Job is being created")
        time.sleep(5)
        continue
    print(response)
    print(json.dumps(response.json(), sort_keys=True, indent=4))
       
    if response.json().get("status") in ["Done","Error", "Canceled"] or response.status_code not in (200,201):
        break
    time.sleep(15)

In [ ]:
# Download job contents
job_id = job_map[action]

endpoint = f"{base_url}/experiments/{data_analytics_experiment_id}/jobs/{job_id}"
response = requests.get(endpoint, headers=headers)
assert response.status_code in (200, 201)
expected_file_size = response.json().get("job_tar_stats", {}).get("file_size")
print("expected_file_size: ", expected_file_size)


endpoint = f'{base_url}/experiments/{data_analytics_experiment_id}/jobs/{job_id}:download'
# Save
temptar = f'{job_id}.tar.gz'

while True:
    # Check if the file already exists
    headers_download_job = dict(headers)
    if os.path.exists(temptar):
        # Get the current file size
        file_size = os.path.getsize(temptar)

        # If the file size matches the expected size, break out of the loop
        if file_size >= expected_file_size:
            print("Download completed successfully.")
            print("Untarring")
            # Untar to destination
            tar_command = f'tar -xf {temptar} -C {workdir}/'
            os.system(tar_command)
            os.remove(temptar)
            print(f"Results at {workdir}/{job_id}")
            inference_out_path = f"{workdir}/{job_id}"
            break

        # Set the headers to resume the download from where it left off
        headers_download_job['Range'] = f'bytes={file_size}-'
    # Open the file for writing in binary mode
    with open(temptar, 'ab') as f:
        try:
            response = requests.get(endpoint, headers=headers_download_job, stream=True)
            # Check if the request was successful
            if response.status_code in [200, 206]:
                # Iterate over the content in chunks
                for chunk in response.iter_content(chunk_size=1024):
                    if chunk:
                        # Write the chunk to the file
                        f.write(chunk)
                        # Flush and sync the file to disk
                        f.flush()
                        os.fsync(f.fileno())
            else:
                print(f"Failed to download file. Status code: {response.status_code}")
        except requests.exceptions.RequestException as e:
            print("Connection interrupted during download, resuming download from breaking point")
            time.sleep(5)  # Sleep for a while before retrying the request
            continue  # Continue the loop to retry the request

### Delete model <a class="anchor" id="head-21"></a>

In [ ]:
endpoint = f"{base_url}/experiments/{annotation_conversion_experiment_id}"

response = requests.delete(endpoint,headers=headers)
assert response.status_code in (200, 201)

print(response)
print(response.json())

In [ ]:
endpoint = f"{base_url}/experiments/{pseudo_mask_experiment_id}"

response = requests.delete(endpoint,headers=headers)
assert response.status_code in (200, 201)

print(response)
print(response.json())

In [ ]:
endpoint = f"{base_url}/experiments/{data_aug_experiment_id}"

response = requests.delete(endpoint,headers=headers)
assert response.status_code in (200, 201)

print(response)
print(response.json())

In [ ]:
endpoint = f"{base_url}/experiments/{data_analytics_experiment_id}"

response = requests.delete(endpoint,headers=headers)
assert response.status_code in (200, 201)

print(response)
print(response.json())

### Delete dataset <a class="anchor" id="head-21"></a>

#### Delete kitti dataset <a class="anchor" id="head-21"></a>

In [ ]:
endpoint = f"{base_url}/datasets/{kitti_dataset_id}"

response = requests.delete(endpoint,headers=headers)
assert response.status_code in (200, 201)

print(response)
print(response.json())

#### Delete coco dataset <a class="anchor" id="head-21"></a>

In [ ]:
endpoint = f"{base_url}/datasets/{coco_dataset_id}"

response = requests.delete(endpoint,headers=headers)
assert response.status_code in (200, 201)

print(response)
print(response.json())

#### Delete coco augment dataset <a class="anchor" id="head-21"></a>

In [ ]:
endpoint = f"{base_url}/datasets/{coco_dataset_augment_id}"

response = requests.delete(endpoint,headers=headers)
assert response.status_code in (200, 201)

print(response)
print(response.json())